# Neural Style Transfer with tf.keras

Оригинальный ноутбук можно скачать по ссылкам:

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Обзор

В этом уроке мы узнаем, как использовать глубокое обучение для создания изображений в стиле образца (вам когда-нибудь хотелось, чтобы вы рисовали, как Пикассо или Ван Гог?). Этот метод известен как **нейронная передача стиля**. Этот метод, описан в [статье Леона А. Гатиса «Нейронный алгоритм художественного стиля» (https://arxiv.org/abs/1508.06576), статья очень полезная, и вы обязательно должны ее посмотреть.

Что такое нейронная передача стиля?

Нейронная передача стиля - это метод оптимизации, используемый для трех изображений: изображения **содержания**, изображения **эталонного стиля** (например, работы известного художника) и **входного** изображения. Нужно смешать их таким образом, чтобы входное изображение выглядело как изображение содержания, но было «нарисовано» в эталонном стиле.


Давайте возьмем изображение этой черепахи и *Великой волны у Канагавы* Кацушика Хокусая:

<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/Green_Sea_Turtle_grazing_seagrass.jpg?raw=1" alt="Drawing" style="width: 200px;"/>
<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/The_Great_Wave_off_Kanagawa.jpg?raw=1" alt="Drawing" style="width: 200px;"/>

[Изображение Зеленой Морской Черепахи] (https://commons.wikimedia.org/wiki/File:Green _Морская_ Черепаха _пасется_ seagrass.jpg)
- П. Линдгрен [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], из общего собрания Викимедиа


Как бы это выглядело, если бы Хокусай решил нарисовать картину этой Черепахи таким стилем? Что-то вроде этого?

<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/wave_turtle.png?raw=1" alt="Drawing" style="width: 500px;"/>

Это какая-то магия или всего лишь глубокое обучение? К счастью, это не колдовство: передача стиля - забавная и интересная техника, которая демонстрирует возможности нейронных сетей.

Принцип нейронной передачи стиля заключается в определении двух разных функций, одна из которых описывает, насколько различается содержимое двух изображений, $L_{content}$, а другая описывает разницу между стилями двух изображений, $L_{style}$. Далее, имея три изображения, желаемое изображение стиля, желаемое изображение содержания и входное изображение, мы пытаемся преобразовать входное изображение так, чтобы минимизировать его отличияот изображения содержания и отличия от изображения эталонного стиля. Мы преобразуем базовое входное изображение, сводя к минимуму расстояния (потери) содержимого и стиля с помощью метода обратного распространения.

### Понятия, рассматриваемые в статье:
В процессе мы будем накапливать практический опыт и развивать интуицию в следующих концепциях:

* Моментальное исполнение (Eager Execution) - использовать императивную среду программирования TensorFlow, которая выполняет операции незамедлительно
    * [Узнайте больше о Eager Execution](https://www.tensorflow.org/programmers _guide / eager)  
    * [Посмотреть его в действии](https://www.tensorflow.org/get_ запущен / готов)
* Использование [Functional API](https://keras.io/getting-started/functional-api-guide/) для определения модели* *- мы создадим подмножество моделей, которое даст нам доступ к необходимым промежуточным функциям активации с помощью Functional API* 
* Использование карт признаков (feature map) предварительно обученной модели* - Узнайте, как использовать предварительно обученнные модели и их карты признаков
* Создание обучающих циклов*- мы рассмотрим, как настроить оптимизатор для минимизации заданных потерь входных параметрам

### Мы должны выполнить следующие шаги для передачи стиля:

1. Визуализация данных
2. Базовая первичная обработка и подготовка данных
3. Настройка функции потерь
4. Создание модели
5. Оптимизация функции потери 

**Аудитория:** Этот пост предназначен для пользователей среднего уровня, которые знакомы с базовыми концепциями машинного обучения. Чтобы получить максимальную пользу от этого поста, вам следует:
* Прочитать [статью Гатиса](https://arxiv.org/abs/1508.06576) - статья предоставит более глубокое понимание задачи
* [Понять уменьшение потерь с помощью градиентного спуска](https://developers.google.com/machine-learning/crash-course/reduc-loss/gradient-descent)

**Расчетное время**: 30 минут

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Первоначальная настройка

### Загрузить изображения

In [ ]:
import os
img_dir = '/tmp/nst'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/d/d7/Green_Sea_Turtle_grazing_seagrass.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg
!wget --quiet -P /tmp/nst/ https://i.ibb.co/WHmm58q/photo-2021-12-15-16-26-22.jpg
!wget --quiet -P /tmp/nst/  https://i.ibb.co/xHJwY4L/photo-2021-12-15-17-09-04.jpg
!wget --quiet -P /tmp/nst/ https://klike.net/uploads/posts/2019-05/1556707966_3.jpg

Импорт модулей и настройка конфигурации

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

Мы начнем с добавления моментального исполнения [(eager execution)](https://www.tensorflow.org/guide/eager). Это позволит нам работать с этой техникой наиболее удобным способом.

In [ ]:
#enable_eager_execution - выполняет операции сразу
# executing_eagerly - проверяет включен ли eager execution
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
# Переменные для изобржения содержания и изображения стиля
content_path = '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg'
style_path = '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg'

**Визуализация изображения**

In [ ]:
def load_img(path_to_img):
    max_dim = 512
    img = Image.open(path_to_img)
    long = max(img.size)
    scale = max_dim/long

#img.size[0] и img.size[1] - ширина и высота
    img = img.resize(
        (round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
#img_to_array - конвертирует изображение PIL в массив Numpy
    img = kp_image.img_to_array(img)

    # We need to broadcast the image array such that it has a batch dimension
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
def imshow(img, title=None):
#  Функция squeeze() удаляет оси с одним элементом (длинной 1), но не сами элементы массива
    # Remove the batch dimension
    out = np.squeeze(img, axis=0)
    # Нормализация отображения
    out = out.astype('uint8')
#imshow - отображение данных в виде изображения
    plt.imshow(out)
    if title is not None:
        plt.title(title)
    plt.imshow(out)

In [ ]:
plt.figure(figsize=(10, 10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Изображение содержания')

plt.subplot(1, 2, 2)
imshow(style, 'Изображение стиля')
plt.show()

## Подготовка данных
Создадим методы, которые позволят нам легко загружать и обрабатывать изображения. Мы выполняем такой же процесс предварительной обработки, который предполагается процессом обучения VGG. Сети VGG обучаются на изображении с каждым каналом, нормализованным по `mean = [103.939, 116.779, 123.68]` и с каналами BGR.

In [ ]:
def load_and_process_img(path_to_img):
    img = load_img(path_to_img)
#tf.keras.applications.vgg19.preprocess_input() - предварительно обрабатывает tensor или массив Numpy
    img = tf.keras.applications.vgg19.preprocess_input(img)
    
    return img

ggg = load_and_process_img(content_path)

Чтобы просмотреть результаты нашей оптимизации, мы должны выполнить шаг обратной предварительной обработки. Кроме того, поскольку наше оптимизированное изображение может принимать значения в промежутке между $- \infty$ и $\infty$, мы должны сократить наши значения до значений в диапазоне 0-255.

In [ ]:
def deprocess_img(processed_img):
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                               "dimension [1, height, width, channel] or [height, width, channel]")
    if len(x.shape) != 3:
        raise ValueError("Invalid input to deprocessing image")

    # выполнить инверсию шага предварительной обработки
#     print(x)
    x[:, :, 0] += 103.939
#     print(x)
    x[:, :, 1] += 116.779
#     print(x)
    x[:, :, 2] += 123.68
#     print(x)
    x = x[:, :, ::-1] # конвертиировать bgr в rgb

    x = np.clip(x, 0, 255).astype('uint8')
    return x

k = deprocess_img(ggg)

### Определение содержания и стиля изображения
Чтобы получить представление о содержании и стиле изображения, мы рассмотрим некоторые промежуточные слои в нашей модели. По мере того, как мы углубляемся в модель, эти промежуточные слои, представляющие собой признаки, становтся более упорядоченными. В этом случае мы используем архитектуру VGG19, которая представляет собой предварительно обученную сеть для классификациии изображений. Промежуточные слои необходимы для определения содержимого и стиля изображений. Для входного изображения мы попытаемся сопоставить соответствующий стиль и содержание на этих промежуточных слоях.

#### Почему промежуточные слои?

Вы можете быть удивлены, почему эти промежуточные слои в предобученной сети классификации изображений позволяют определить стиль и содержание изображения. Это явление может быть объяснено тем фактом, что для того, чтобы сеть выполняла классификацию изображений (чему сеть уже была обучена), она должна понимать изображение. Это включает в себя использование необработанного изображения в виде входных пикселей и построение представления посредством преобразований, которые превращают необработанные пиксели изображения в сложные объекты, присутствующие в изображении. Это также частично объясняет, почему сверточные нейронные сети способны хорошо обобщать: они способны фиксировать постоянство и определять особеннности, характерные, какому либо классу (например, кошки и собаки), не зависящие от фонового шума. Таким образом, где-то между тем, где подается необработанное изображение и выводится результат классификации, модель служит в качестве экстрактора сложных признаков. Следовательно, получая доступ к промежуточным слоям, мы можем описать содержание и стиль входных изображений.


В частности, мы извлечем эти промежуточные слои из нашей сети:

In [ ]:
# Слой содержания, в который помещается карта объектов
content_layers = ['block5_conv2']

# Интересный нам слой стиля
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1'
                ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

## Построение модели
Мы должны загрузить [VGG19] (https://keras.io/applications/#vgg19) и ввести входной тензор в модель. Это позволит нам получить карты признаков (и далее содержание и стиль) изображения.

Мы используем VGG19, как предлагается в статье. Кроме того, поскольку VGG19 является относительно простой моделью (по сравнению с ResNet, Inception и т. Д.), карты признаков работают лучше для передачи стилей.

Чтобы получить доступ к промежуточным слоям, соответствующим картам признаков стиля и содержимого, мы должны получить соответствующие выходные данные,используя Keras [**functional API**](https://keras.io/getting-started/ function-api-guide /), мы определяем нашу модель с желаемыми выходными функциями активациями.

С помощью functional API определение модели сводится к определению входных и выходных данных:

`модель = модель (входы, выходы)`

In [ ]:
def get_model():
    """ Создание модели с доступом к промежуточным слоям.
  
    Эта функция будет загружать модель VGG19 и давать доступ к промежуточным слоям.
    В дальнейшем эти слои будут использоваться для создания модели, которая будет брать входное изображение
    и возвращать выходные данные с промежуточных слоёв модели VGG.
    """
    # Загрузка нашей модели. Мы загружаем предобученную модель VGG
    vgg = tf.keras.applications.vgg19.VGG19(
        include_top=False, weights='imagenet')
    vgg.trainable = False
    # Получение слоев стиля и содержания
    style_outputs = [vgg.get_layer(name).output for name in style_layers]
    content_outputs = [vgg.get_layer(name).output for name in content_layers]
    model_outputs = style_outputs + content_outputs
    # Построение модели
    return models.Model(vgg.input, model_outputs)

В приведенном выше фрагменте кода мы загрузим предварительно подготовленную сеть классификации изображений. Затем мы возьмем необходимые слои, как мы говорили ранее. Затем мы определяем модель, устанавливая входные данные для изображения, и выходные данные для выходных слоев стиля и содержимого. Другими словами, мы создали модель, которая будет брать входное изображение и выводить промежуточные слои содержимого и стиля.

## Определение и создание функции потерь (расстояний содержания и стиля)

### Функция потерь содержания

Наше определение функции потери содержания довольно простое. Мы передадим сети желаемое изображение содержания и наше базовое входное изображение. После мы получим выходные данные промежуточного слоя из нашей модели. Затем мы просто берем евклидово расстояние между двумя промежуточными представлениями этих изображений.

Более формально, функция потери содержимого - это функция, которая описывает расстояние содержимого от входного изображения $x$ и изображения содержимого $p$. Пусть $C_{nn}$ будет предварительно обученной глубокой сверточной нейронной сетью. Опять же, в этом случае мы используем [VGG19] (https://keras.io/applications/#vgg19). Пусть $X$ будет любым изображением, тогда $C_{nn}(X)$ - это сеть, на вход которой подается X. Пусть $F^l_{ij}(x) \in C_{nn}(x)$ и $P^l_{ij}(p) \in C_{nn}(p)$ описывают соответствующее промежуточное представление функции сети, принимающей на вход $x$ и $p$ на уровне $l$. Затем мы рассчитываем расстояние содержимого (потери) как: $$L^l_{content}(p, x) = \sum_{i, j} (F^l_{ij}(x) - P^l_{ij}(p))^2$$

Мы выполняем обратное распространение, чтобы минимизировать потерю содержимого. Таким образом, мы изменяем исходное изображение до тех пор, пока оно не сгенерирует ответ в определенном слое (определенный в content_layer) как исходное изображение контента.

Это можно реализовать довольно просто. На вход нужно подать карту признаков слоя L в сети, со входом x, входное изображение, и p, изображение контента, и вернуть расстояние содержания.

### Вычисление функции потери содержания
Мы добавим функцию потери содержания на каждом желаемом слое. Таким образом, при каждой итерации передачи входного изображение в модель, все потери содержимого в модели будут правильно вычисляться, и, поскольку мы используем моментальное исолнение, все градиенты будут вычислены.

In [ ]:
def get_content_loss(base_content, target):
    return tf.reduce_mean(tf.square(base_content - target))

## Функция потерь стиля

Вычисление потери стиля немного сложнее, но базируется на том же принципу, но на этот раз в нашу сеть подается базовое входное изображение и изображение стиля. Однако вместо сравнения необработанных промежуточных выходных данных основного входного изображения и изображения стиля мы сравниваем матрицы Грамма двух выходных данных.

Математически мы описываем потерю стиля основного входного изображения, $x$, и изображения стиля, $a$, как расстояние между представлением стиля (матрица Грама) этих изображений. Мы описываем представление стиля изображения как корреляцию между различными ответами фильтра, заданными матрицей Грама $G^l$, где $G^l_{ij}$ является внутренним произведением между векторизованной картой признаков $i$ и $j$ в слое $l$.

Чтобы сгенерировать стиль для основного входного изображения, мы выполняем градиентный спуск от изображения содержимого для чтобы преобразовать его в изображение, соответствующее стилевому представлению исходного изображения. Мы делаем это путем минимизации среднего квадрата расстояния между картой корреляции изображения стиля и входного изображения. Вклад каждого слоя в общую потерю стиля описывается
$$E_l = \frac{1}{4N_l^2M_l^2} \sum_{i,j}(G^l_{ij} - A^l_{ij})^2$$

где $G^l_{ij}$ и $A^l_{ij}$ - соответствующее представление стиля в слое $l$ из $x$ и $a$. $N_l$ описывает количество карт объектов, каждая из которых имеет размер $M_l = height * width$. Таким образом, общая потеря стиля на каждом слое составляет
$$L_{style}(a, x) = \sum_{l \in L} w_l E_l$$
где взвешивается влияние потери каждого слоя на некоторый коэффициент $w_l$. В нашем случае мы взвешиваем каждый слой одинаково ($w_l =\frac{1}{|L|}$)

### Вычисление функции потерь стиля

In [ ]:
def gram_matrix(input_tensor):
    # Сначала идёт канал изображения
    channels = int(input_tensor.shape[-1])
    print(input_tensor.shape)
    a = tf.reshape(input_tensor, [-1, channels])
    print(a.shape)
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)


def get_style_loss(base_style, gram_target):
    """Принимает два изображения измерений h, w, c"""
    # Высота, ширина и количество фильтров в каждом слое
    # Масштабируем потерю в данном слое
    height, width, channels = base_style.get_shape().as_list()
    gram_style = gram_matrix(base_style)

    # / (4. * (channels ** 2) * (width * height) ** 2)
    return tf.reduce_mean(tf.square(gram_style - gram_target))

## Применение передачи стиля к нашим изображениям

### Запуск градиентного спуска
Если вы не знакомы с градиентным спуском/обратным распространением или нуждаетесь в переподготовке, вам обязательно нужно перейти на этот [потрясающий ресурс](https://developers.google.com/machine-learning/crash-course/reduc-loss/gradient -descent).

В этом случае мы юудем использовать оптимизатор [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam), чтобы минимизировать потери. Мы итеративно обновляем наш выходное изображение: мы не обновляем веса в сети, вместо этого мы обучаем входное изображение, чтобы минимизировать потери. Для того, что это сделать, мы должны знать, как рассчитывать потери и градиенты.

\* Обратите внимание, что c помощью L-BFGS, который не рекомендуется использовать в этом учебном пособии, поскольку основная мотивация этого учебного пособия заключалается в том, чтобы проиллюстрировать лучшие практики с мгновенным выполением, и с помощью Adam мы можем продемонстрировать функциональность autograd/gradient tape с помощью пользовательских циклов обучения.

Мы определим небольшую вспомогательную функцию, которая будет загружать изображение содержания и изображение стиля, передавать их в сеть, которая затем будет выводить представления функций содержания и стиля нашей модели.

In [ ]:
def get_feature_representations(model, content_path, style_path):
    """
    Функция, которая рассчитывает признаки стиля и контента.
 
    Эта функция будет просто предварительно подгружать и обрабатывать содержимое и стиль изображений. 
    Затем эти представления пройдут через сеть, чтобы получить промежуточные слои.
  
    Аргументы:
      model: Используемая модель.
      content_path: Путь к изображению содержимого.
      style_path: Путь к изображению стиля.
    
    Возвращает:
      Признаки стиля и контента.
    """
    # Загрузка изображений
    content_image = load_and_process_img(content_path)
    style_image = load_and_process_img(style_path)

    # Обработка признаков стиля и содержания
    style_outputs = model(style_image)
    content_outputs = model(content_image)

    # Получение представлений признаков содержания стиля и содержания из модели
    style_features = [style_layer[0]
                      for style_layer in style_outputs[:num_style_layers]]
    content_features = [content_layer[0]
                        for content_layer in content_outputs[num_style_layers:]]
    print(style_features)
    return style_features, content_features

### Вычисление функции потерь и градиентов
Здесь мы используем [**tf.GradientTape**] (https://www.tensorflow.org/programmers _guide / eager # computing_ градиенты) для вычисления градиента. Это позволяет нам использовать преимущества автоматического дифференциирования, доступного благодаря отслеживанию операций для вычисления градиента. Он записывает операции во время прямого прохода и затем может вычислить градиент функции потерь относительно нашего входного изображения для обратного прохода.

In [ ]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
    """Эта функция рассчитывает полную потерю.
  
    Аргументы:
      model: Модель с нужными промежуточными слоями.
      loss_weights: Вес каждого компонента для каждой функции потерь. 
        (вес для стиля, для содерджания и общий).
      init_image: Первичное изображение. Это то изображение, которое в процессе оптимизации будет обновляться.
      gram_style_features: Предварительные вычисления матрицы Грама соответствующих слоёв.
      content_features: Предварительные вычисления нужных слоёв контента.
      
    Возвращает:
      Общие потери, потери для стиля, содержания и вариационные потери
    """
    style_weight, content_weight = loss_weights

    # Прогон изображение через модель. Это даст представления содержания и стиля.
    # Из-за использования мгновенного выполнения, эта модель вызывается как и любая другая функция.
    model_outputs = model(init_image)

    style_output_features = model_outputs[:num_style_layers]
    content_output_features = model_outputs[num_style_layers:]

    style_score = 0
    content_score = 0

    # Суммирует потерю стиля со всех слоёв
    # Тут одинаково взвешиваются потери каждого слоя.
    weight_per_style_layer = 1.0 / float(num_style_layers)
    for target_style, comb_style in zip(gram_style_features, style_output_features):
        style_score += weight_per_style_layer * \
            get_style_loss(comb_style[0], target_style)

    # Суммирование потерь контента со всех слоёв
    weight_per_content_layer = 1.0 / float(num_content_layers)
    for target_content, comb_content in zip(content_features, content_output_features):
        content_score += weight_per_content_layer * \
            get_content_loss(comb_content[0], target_content)

    style_score *= style_weight
    content_score *= content_weight

    # Получение суммарной потери
    loss = style_score + content_score
    return loss, style_score, content_score

В итоге расчёт градиента сводится к этому:

In [ ]:
def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(**cfg)
    # Расчёт градиента изображения
    total_loss = all_loss[0]
    return tape.gradient(total_loss, cfg['init_image']), all_loss

### Оптимизация

In [ ]:
import IPython.display

def run_style_transfer(content_path, 
                       style_path,
                       num_iterations=1000,
                       content_weight=1e3, 
                       style_weight=1e-2): 
  # В этом случае не нужно обучать каждый слой модели. Поэтому параметр trainability нужно выставить в false.
  model = get_model() 
  for layer in model.layers:
    layer.trainable = False
  
  # Получение представлений признаков стиля и контента (из промежуточных слоёв).
  style_features, content_features = get_feature_representations(model, content_path, style_path)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  
  # Загрузка исходного изображения
  init_image = load_and_process_img(content_path)
  init_image = tf.Variable(init_image, dtype=tf.float32)
  # Создание оптимизатора
  opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)

  # Отображение промежуточных изображений
  iter_count = 1
  
  # Сохранение лучшего результата
  best_loss, best_img = float('inf'), None
  
  # Создание конфигурации
  loss_weights = (style_weight, content_weight)
  cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features
  }
    
  # Отображение
  num_rows = 2
  num_cols = 5
  display_interval = num_iterations/(num_rows*num_cols)
  start_time = time.time()
  global_start = time.time()
  
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   
  
  imgs = []
  for i in range(num_iterations):
    grads, all_loss = compute_grads(cfg)
    loss, style_score, content_score = all_loss
    opt.apply_gradients([(grads, init_image)])
    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time() 
    
    if loss < best_loss:
      # Обновление лучшей потери и изображения 
      best_loss = loss
      best_img = deprocess_img(init_image.numpy())

    if i % display_interval== 0:
      start_time = time.time()
      
      # Используйте метод .numpy(), чтобы получить конкретный numpy-массив
      plot_img = init_image.numpy()
      plot_img = init_image.numpy()
      plot_img = deprocess_img(plot_img)
      imgs.append(plot_img)
      IPython.display.clear_output(wait=True)
      IPython.display.display_png(Image.fromarray(plot_img))
      print('Iteration: {}'.format(i))        
      print('Total loss: {:.4e}, ' 
            'style loss: {:.4e}, '
            'content loss: {:.4e}, '
            'time: {:.4f}s'.format(loss, style_score, content_score, time.time() - start_time))
  print('Total time: {:.4f}s'.format(time.time() - global_start))
  IPython.display.clear_output(wait=True)
  plt.figure(figsize=(14,4))
  for i,img in enumerate(imgs):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(img)
      plt.xticks([])
      plt.yticks([])
      
  return best_img, best_loss 

In [ ]:
best, best_loss = run_style_transfer(content_path,
                                     style_path, num_iterations=1000)

In [ ]:
Image.fromarray(best)

Для того, чтобы загрузить изображение из Colab, расскоментируйте:

In [ ]:
#from google.colab import files
#files.download('wave_turtle.png')

## Визуализация результатов

In [ ]:
def show_results(best_img, content_path, style_path, show_large_final=True):
    plt.figure(figsize=(10, 5))
    content = load_img(content_path)
    style = load_img(style_path)

    plt.subplot(1, 2, 1)
    imshow(content, 'Изображение содержания')

    plt.subplot(1, 2, 2)
    imshow(style, 'Изображение стиля')

    if show_large_final:
        plt.figure(figsize=(10, 10))

        plt.imshow(best_img)
        plt.title('Выходное изображение')
        plt.show()

In [ ]:
show_results(best, content_path, style_path)

## Попробуем на других изображениях
Изображение Тюбингена

Фото: Андреас Праефке [GFDL (http://www.gnu.org/copyleft/fdl.html) или CC BY 3.0 (https://creativecommons.org/licenses/by/3.0)], взято из Викисклада.

### Звездная ночь + Тюбинген

In [ ]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [ ]:
show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [ ]:
!ls -la /tmp/nst

### Песик 1 + Ван Гог

In [ ]:
best_starry_night, best_loss = run_style_transfer("/tmp/nst/photo-2021-12-15-16-26-22.jpg",
                                                  "/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg")



In [ ]:
show_results(best_starry_night, '/tmp/nst/photo-2021-12-15-16-26-22.jpg',
             '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

### Песик 2 + Кандинский

In [ ]:
best_kandinsky_tubingen, best_loss = run_style_transfer('/tmp/nst/photo-2021-12-15-17-09-04.jpg', 
                                                  '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

In [ ]:
show_results(best_kandinsky_tubingen, 
             '/tmp/nst/photo-2021-12-15-17-09-04.jpg',
             '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

### Волна

In [ ]:
best_poc_turtle, best_loss = run_style_transfer('/tmp/nst/1556707966_3.jpg', 
                                                  '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg')

In [ ]:
show_results(best_poc_turtle, 
             '/tmp/nst/1556707966_3.jpg',
             '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg')

### Столпы Творения + Тюбинген

In [ ]:
best_poc_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [ ]:
show_results(best_poc_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

### Кандинский Композиция 7 + Тюбинген

In [ ]:
best_kandinsky_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg', 
                                                  '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

In [ ]:
show_results(best_kandinsky_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

### Столпы Творения + Морская Черепаха

In [ ]:
best_poc_turtle, best_loss = run_style_transfer('/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg', 
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [ ]:
show_results(best_poc_turtle, 
             '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [ ]:
best_poc_turtle, best_loss = run_style_transfer('/tmp/nst/The_Great_Wave_off_Kanagawa.jpg', 
                                                  '/tmp/nst/1556707966_3.jpg')

In [ ]:
show_results(best_poc_turtle, 
             '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg',
             '/tmp/nst/1556707966_3.jpg')

## Ключевые результаты

### Что мы сделали:

*Мы создали несколько различных функций потерь и использовали метод обратного распространения для преобразования входного изображения, чтобы минимизировать потери*. Для этого нам пришлось загрузить **предварительно обученную модель** и использовать карты прихнаков для описания содержания и стиля изображений.
    *Функции потерь в основном вычисляли расстояния различных представлений*. Мы реализовали это с помощью собственной модели и **моментального исполнения**.
  *Мы создали собственную модель с помощью Functional API*. Моментальное исполнение позволяет нам динамически работать с тензорами, используя естественный поток управления Python.
  *Мы напрямую управляли тензорами, что облегчило отладку и работу с тензорами.* Мы итеративно обновляли наше изображение, применяя правила обновления оптимизаторов с использованием **tf.gradient**. Оптимизатор минимизировал потери относительно входного изображения.


**[Image of Tuebingen](https://commons.wikimedia.org/wiki/File:Tuebingen_Neckarfront.jpg)** 
Photo By: Andreas Praefcke [GFDL (http://www.gnu.org/copyleft/fdl.html) or CC BY 3.0  (https://creativecommons.org/licenses/by/3.0)], from Wikimedia Commons

**[Image of Green Sea Turtle](https://commons.wikimedia.org/wiki/File:Green_Sea_Turtle_grazing_seagrass.jpg)**
By P.Lindgren [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], from Wikimedia Commons

